In [2]:
search_for = 137
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002792835235595703
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 354865621
type: [1, 1, 1, 1, 137] 137
cases of this type: 352275361
100000 18.89226934909075
200000 33.47753985835563
300000 33.29080840552015
400000 33.43205713347558
500000 32.91385250331111
600000 32.98234637129464
700000 34.44174370641138
800000 33.019905447451805
900000 32.59274591624315
1000000 32.76122201854879
1100000 32.327593324414075
1200000 32.6043952169442
1300000 15.782354687963819
1400000 24.129804529800666
1500000 32.43043062284708
1600000 32.414799290413
1700000 32.27648244464278
1800000 32.312619472665034
1900000 32.824705268998585
2000000 33.97567290104211
2100000 32.52903005590388
2200000 32.13884885304602
2300000 32.617301978284026
2400000 33.34421506976696
2500000 32.71625891435975
2600000 12.806560919021392
2700000 28.18090447764257
2800000 40.98604451885206
2900000 41.75419977957542
3000000 40.551360256302

29800000 30.566277314695174
29900000 31.458927623620895
30000000 32.06930384885496
30100000 32.083116238663465
30200000 31.796400792551005
30300000 32.27054752914061
30400000 34.092207987771886
30500000 32.66330233518124
30600000 31.863487563039264
30700000 12.628836473339433
30800000 34.54956541807814
30900000 33.98763778686384
31000000 33.845057828065414
31100000 20.28754686567821
31200000 24.30881628782893
31300000 33.72521957689721
31400000 34.117952356266606
31500000 33.59216089391013
31600000 32.705069148609745
31700000 34.14837518512389
31800000 33.12678021824304
31900000 32.03478670465454
32000000 32.143585573489375
32100000 30.685889505722233
32200000 7.42656076275564
32300000 32.377698946027394
32400000 31.316704977435535
32500000 31.874831850700946
32600000 32.62104144730842
32700000 32.63860978144433
32800000 31.4620434562166
32900000 33.072938793574046
33000000 33.11861489763419
33100000 32.48355105849913
33200000 20.50413764732252
33300000 24.741365806316967
33400000 33.1

59600000 15.846755416683196
59700000 23.956884362093415
59800000 29.53044562831113
59900000 28.02643234910105
60000000 28.55028338940283
60100000 27.037948883275405
60200000 26.640702622506996
60300000 26.332939457172152
60400000 22.865004285794896
60500000 11.52747428512267
60600000 26.201558199645504
60700000 26.711776446819165
60800000 26.449243768754386
60900000 27.48579322633065
61000000 27.86589165545963
61100000 27.882465258936904
61200000 28.408570791148495
61300000 13.592187198503368
61400000 27.315703645577248
61500000 29.450546814479605
61600000 28.820092116719128
61700000 30.27348614560816
61800000 29.8823726459449
61900000 29.912588344802995
62000000 30.335938441840778
62100000 30.713486043654395
62200000 13.656694110724084
62300000 26.29944035327205
62400000 29.198340264525967
62500000 28.709869483682983
62600000 29.4959434392346
62700000 27.70861755715234
62800000 27.246936358745803
62900000 26.775970619406028
63000000 16.255065680607515
63100000 18.188023411850047
63200

89300000 24.55607582250325
89400000 8.545842306673077
89500000 26.847011257458437
89600000 25.75466933682046
89700000 26.046676925160746
89800000 26.564228793417815
89900000 26.227793207093967
90000000 27.666308517306657
90100000 25.436597705028127
90200000 26.461569217447465
90300000 26.21939630974507
90400000 25.742515805580336
90500000 25.211549712914856
90600000 25.63167664961259
90700000 9.706572918538114
90800000 22.1320006575324
90900000 24.002451331240767
91000000 22.823910930690264
91100000 23.371402639837097
91200000 23.67645286260928
91300000 11.05857695063596
91400000 18.451264256882425
91500000 23.759229891845514
91600000 23.233534865254303
91700000 23.193512584224973
91800000 23.392426362270573
91900000 16.56348446970039
92000000 17.85188179469143
92100000 24.85605126992869
92200000 24.806576988678987
92300000 26.13282434264881
92400000 25.362712378607302
92500000 26.368889154824274
92600000 25.47225275389491
92700000 26.475954151983856
92800000 26.537631312463503
9290000

118400000 24.597167716998534
118500000 23.552265192193204
118600000 23.96489296743606
118700000 23.91604655731169
118800000 22.652224398159937
118900000 22.17223717168131
119000000 21.680545829302304
119100000 21.809247391193658
119200000 6.596360669498401
119300000 21.83387044134771
119400000 22.081559319360437
119500000 21.23078700186404
119600000 7.332478705737342
119700000 19.88615832539349
119800000 21.406223571107382
119900000 21.579719960337016
120000000 11.75338216532459
120100000 18.284136235607832
120200000 21.400978573723958
120300000 21.10449479496862
120400000 21.686727667277168
120500000 22.138325549414997
120600000 22.803067320089415
120700000 23.50351939099479
120800000 22.927369395832383
120900000 23.358910657725193
121000000 23.285157978688897
121100000 22.887421038582524
121200000 22.502260045962686
121300000 22.75606396188658
121400000 21.265828107336205
121500000 21.054211787846164
121600000 20.66069802732856
121700000 21.308934557305268
121800000 6.422255804013832

...

233000000 11.861902909768174
233100000 9.419892333627885
233200000 5.796998059321241
233300000 11.641715180780466
233400000 11.442006696225057
233500000 12.056610101409047
233600000 11.864535582547095
233700000 12.269012412680743
233800000 12.565232478448692
233900000 12.485219629186874
234000000 12.630010618582588
234100000 11.997354795481817
234200000 12.142228610555952
234300000 11.92092458824486
234400000 11.76177191597846
234500000 11.07327096273757
234600000 11.310854469339654
234700000 10.774394108382804
234800000 11.109061250992603
234900000 3.951424394476781
235000000 11.273212109186703
235100000 10.999723525283956
235200000 10.878080687501528
235300000 4.678354140946615
235400000 9.126172176941449
235500000 11.329957006682237
235600000 11.023079920265559
235700000 8.236856498296577
235800000 6.778278484275041
235900000 10.708511853600406
236000000 11.089145297156309
236100000 11.566573921634653
236200000 11.383825349366749
236300000 12.098093147005224
236400000 13.24101144265

262000000 9.29672489124239
262100000 9.307646565912993
262200000 9.766576881325117
262300000 8.810067849952114
262400000 9.277668392743669
262500000 9.21230258898416
262600000 9.22290471496906
262700000 9.044104011319494
262800000 8.846053127622943
262900000 7.720502950977747
263000000 4.52477036461361
263100000 8.304779261087079
263200000 8.459568236033892
263300000 8.269073776501886
263400000 8.311400438592763
263500000 8.633587135679466
263600000 2.43512101481018
263700000 8.146551864983193
263800000 8.574421986327756
263900000 8.589384430941635
264000000 8.438951003732218
264100000 8.56804570888824
264200000 5.745263637861415
264300000 5.906995187315857
264400000 9.100754499035347
264500000 9.032639203024768
264600000 9.280194570539692
264700000 9.254911912816992
264800000 9.548960799187908
264900000 9.63998237486897
265000000 9.335865038923156
265100000 8.911136168490167
265200000 9.197397106152941
265300000 8.821236846179294
265400000 8.662626097755654
265500000 4.500866089309069

291300000 5.96565950757928
291400000 5.9545231772366005
291500000 5.776704189843351
291600000 5.751808877071811
291700000 5.650409671400025
291800000 5.864903570899373
291900000 1.1215099712278127
292000000 5.908769158804386
292100000 5.7488629230419335
292200000 5.686726827293989
292300000 5.672890045630183
292400000 6.014034272219292
292500000 5.939413379476774
292600000 6.039486859942308
292700000 3.804041927260164
292800000 4.54745817710187
292900000 6.276460948943245
293000000 6.243719940317611
293100000 6.359388651399905
293200000 6.306764931476382
293300000 6.295932210497999
293400000 6.2083260452216615
293500000 6.261169287636706
293600000 2.0966482818590726
293700000 6.564609361963343
293800000 6.113706127165842
293900000 5.882135883870271
294000000 6.000014859300737
294100000 5.719648868028891
294200000 5.7272954902075774
294300000 5.619050925241734
294400000 4.410516081525585
294500000 2.735358779276414
294600000 5.5377710729364225
294700000 5.6243231072475375
294800000 5.70

320400000 3.452980452636357
320500000 3.5617954354801578
320600000 3.5684282230411632
320700000 3.6892411459860894
320800000 3.496035336146647
320900000 3.6141580801284867
321000000 3.606160531331333
321100000 3.5908789569905877
321200000 1.9526435526340344
321300000 2.8253213902660885
321400000 3.664714919997662
321500000 3.4229024242819315
321600000 3.2379258150157146
321700000 1.4725947752329052
321800000 3.4184292502942775
321900000 3.324204131138929
322000000 3.507429080344534
322100000 3.2302724053813088
322200000 3.1954295497267404
322300000 3.3436154215078155
322400000 3.243307710284183
322500000 3.1952080616084992
322600000 3.085734808273476
322700000 2.0742748494840995
322800000 1.9159790623841153
322900000 3.075702221625412
323000000 3.1883327482497466
323100000 3.2012010138220073
323200000 3.2467109946772514
323300000 3.175983989885049
323400000 3.1675270949629524
323500000 3.2727121901987433
323600000 3.284660805344463
323700000 3.270525835765762
323800000 1.52209394889061

349000000 0.5576181019316614
349100000 0.5618213457838238
349200000 0.5510659088396106
349300000 0.5644033126191695
349400000 0.5544188530146911
349500000 0.4955808827016109
349600000 0.49239437429578903
349700000 0.19569792817642953
349800000 0.21549368060838778
349900000 0.42305666203403475
350000000 0.38656780104148847
350100000 0.36101450013144676
350200000 0.38817058466827475
350300000 0.329461126938706
350400000 0.3580918527512133
350500000 0.30975782843906285
350600000 0.3417474601741367
350700000 0.31736144137442446
350800000 0.3349189169323425
350900000 0.2799500868841582
351000000 0.13233404588068523
351100000 0.210656520680515
351200000 0.2852911143696361
351300000 0.24266495824090176
351400000 0.27635223955675364
351500000 0.19093571435165538
351600000 0.22183297606533167
351700000 0.2534482587168972
351800000 0.18605523162955576
351900000 0.1708604161833677
352000000 0.22660553952168622
352100000 0.16949677480455172
352200000 0.13771347120500338
best so far: 0
type: [1, 1,